In [1]:
import torch, torchvision
import torch.nn as nn

import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt

import detectron2
import detectron2.data.transforms as T
from detectron2.data.transforms import Transform
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog, DatasetMapper, build_detection_train_loader
from detectron2.modeling import build_model,build_resnet_backbone,build_backbone
from detectron2.structures import ImageList, Instances
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling.meta_arch.rcnn import GeneralizedRCNN
from detectron2.evaluation import COCOEvaluator
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

- Contrast normalization: The myeloma cells have varying levels of contrast as compared to other cells and tissues, so normalizing the contrast can help them be more visible
- Morphological operations: Erosion or dilation operations can smooth the edges of the cells can help us detect the cancer cells better
- Gradient Filters: Sobel filter can help identify the boundaries of the cells better
- Color Channels: Manipulate the different color channels (RGB) of the image by suppressing or enhancing the effect of either red, green, or blue channel
- Blur Filter: Try the Gaussian blur filters to smooth the image and reduce the noise
- Resolution: Benchmark the accuracy of detecting myeloma cells by reducing the resolution of the image to see if we can get at par accuracy with a smaller dimension image

In [ ]:
# data.transforms.ColorTransform
# data.transforms.RandomBrightness
# data.transforms.RandomContrast
# data.transforms.RandomSaturation
# data.transforms.RandomLighting

# ColorTransform(brightness_delta=20, contrast_range=(0.8, 1.2), saturation_range=(0.8, 1.2))

In [ ]:
class CorrectColor(Transform):
    """
    Color correct the input image to RGB format
    """
    def __init__(self):
        super().__init__()

    def apply_image(self, img):
        ## Convert image to RGB format
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img

    def apply_coords(self, coords):
        # This transform does not modify the bounding box coordinates
        return coords

    def get_transform(self, image):
        # This transform does not depend on the input image
        return self


In [ ]:
class GaussianBlur(Transform):
    """
    Apply a Gaussian blur to the input image.
    """
    def __init__(self, kernel_size=(15, 15), sigma=0.0):
        super().__init__()
        self.kernel_size = kernel_size
        self.sigma = sigma

    def apply_image(self, img):        
        ## apply Gaussian kernel
        img = cv2.GaussianBlur(img, self.kernel_size, self.sigma)
        return img

    def apply_coords(self, coords):
        # This transform does not modify the bounding box coordinates
        return coords

    def get_transform(self, image):
        # This transform does not depend on the input image
        return self


In [ ]:
# abc = CorrectColor()
# im = cv2.imread("../TCIA_SegPC_dataset/coco/x/106.bmp")
# im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
# plt.imshow(im);

In [ ]:
# im1 = abc.apply_image(im)
# plt.imshow(im1);

In [ ]:
def build_sem_seg_train_aug(cfg):
    """
    Define a list of augmentations to apply to the images.
    """
    
    augs= [CorrectColor(), GaussianBlur()]
    
    return augs

In [ ]:
register_coco_instances("SegPC_train", {}, "../TCIA_SegPC_dataset/coco/COCO.json", "../TCIA_SegPC_dataset/coco/x/")
register_coco_instances("SegPC_val", {}, "../TCIA_SegPC_dataset/coco_val/COCO.json", "../TCIA_SegPC_dataset/coco_val/x/")

train_meta = MetadataCatalog.get('SegPC_train')
val_meta = MetadataCatalog.get('SegPC_val')

train_dicts = DatasetCatalog.get("SegPC_train")
val_dicts = DatasetCatalog.get("SegPC_val")

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml"))
cfg.DATASETS.TRAIN = ("SegPC_train",)
cfg.DATASETS.TEST = ("SegPC_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("Misc/cascade_mask_rcnn_X_152_32x8d_FPN_IN5k_gn_dconv.yaml")  
cfg.SOLVER.IMS_PER_BATCH = 3
cfg.SOLVER.BASE_LR = 0.02/8
cfg.SOLVER.LR_SCHEDULER_NAME = 'WarmupCosineLR'

cfg.SOLVER.WARMUP_ITERS = 1500
cfg.SOLVER.MAX_ITER = 2000
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.CHECKPOINT_PERIOD = 1000

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.TEST.EVAL_PERIOD = 250

cfg.CUDNN_BENCHMARK = True
cfg.OUTPUT_DIR = "./output/"

In [ ]:
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
            
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        
        mapper = DatasetMapper(cfg, is_train=True, augmentations=build_sem_seg_train_aug(cfg))
        
        return build_detection_train_loader(cfg, mapper=mapper)

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
# from detectron2.utils.visualizer import Visualizer, ColorMode

# for d in random.sample(train_dicts, 3):
#     print(d["file_name"])
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=train_meta, instance_mode=ColorMode.IMAGE_BW, scale=1)
#     out = visualizer.draw_dataset_dict(d)
#     plt.imshow(out.get_image()[:, :, ::-1])